<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

<a href="http://www.hupi.fr/" ><img src="http://www.hupi.fr/wp-content/uploads/2016/03/hupi_logo_vectoris_menu.png" style="max-width: 300px; display: inline"  alt="Wikistat"/></a>

<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" style="float:right; max-width: 200px; display: inline" alt="IMT"/> </a>
</center>


________________________________________________________________________________________________________________________________

________________________________________________________________________________________________________________________________


# Données de site internet en streaming - Streaming on internet website


________________________________________________________________________________________________________________________________

________________________________________________________________________________________________________________________________

## Sujet - Subject

Cette fois-ci, nous allons récupérer en streaming les données de l'usine ecoles (http://ecoles.hupi.io/). Nous récupérerons les données et afficheront le nombre d'utilisateurs sur le site et le nombre d'actions (clics,...).
> 
*
Here, we are going to collect in streaming the data of the ecoles platform (http://ecoles.hupi.io/). We will collect the data and display the number of users on the website and the number of actions (clicks,...).
*

## Importation des libraries - Library import

In [1]:
# Modules pour la liaison entre Kafka et Spark / Packages for linking Kafka et Spark
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

## SparkContext

In [2]:
# Si il n'y a pas de SparkContext, il faut le créer / Create a SparkContext if there is not :
#from pyspark import SparkContext # Si il n'existe pas déjà / if it doesn't exist
#sc = SparkContext(appName="PythonStreamingKafka_SentimentAnalysis")

## Liaison avec Kafka - Link with Kafka

In [3]:
intervalle = 10 # Fenêtre de x secondes / x seconds window

ssc = StreamingContext(sc, intervalle) 

zkQuorum = "hupi.node1.pro.hupi.loc:2181"  

topic = {"hupi_hupilytics": 1} 

identifiant = "Anthony" # A modifier pour chaque personne / to modify for each user

streamdata = KafkaUtils.createStream(ssc, zkQuorum, identifiant, topic)

## Traitement - Treatment

In [4]:
# Fonction d'affichage / Display fonction
def get_output(rdd):
    li = rdd.collect()
    for x in li:
        print x
    print("\n")

In [5]:
# Affichage des résultats / Display results

intro = streamdata.count().map(lambda l : "\n   Résultats pour cet interval : \n")
intro.pprint()


actions = streamdata.count().map(lambda l : " - Nombre d'actions sur le site : "+str(l))
actions.foreachRDD(lambda w : get_output(w))

userss = streamdata.count().map(lambda l : " - ID des utilisateurs par action : ")
userss.foreachRDD(lambda w : get_output(w))
nb_user = streamdata.map(lambda l: l[1]) \
                   .map(lambda rdd : ("      --> ID " + \
                                      str(rdd[(rdd.find("\"_id\"")+7):(rdd.find("idts")-4)]) ) )
nb_user.foreachRDD(lambda w : get_output(w))

### Lancement du programme - Program's launching

In [6]:
ssc.start()
ssc.awaitTermination(intervalle)

-------------------------------------------
Time: 2016-05-31 10:05:00
-------------------------------------------

   Résultats pour cet interval : 


 - Nombre d'actions sur le site : 5


 - ID des utilisateurs par action : 


      --> ID 5b384ce32d8cdef0
      --> ID 4d89d294cd4ca9f2
      --> ID 4d89d294cd4ca9f2
      --> ID 5b384ce32d8cdef0
      --> ID 5b384ce32d8cdef0


-------------------------------------------
Time: 2016-05-31 10:05:10
-------------------------------------------









In [7]:
ssc.stop(stopSparkContext=False) # Stop streaming